# 特徵處理

當然除了這些還有其他方法, 參考 012.特徵相關.

1. PCA : 降維
2. AutoFeat : 升維
3. 相關性

## PCA(主成分分析) : 降維

用來降低維度, 且保留數據的主要訊息.

In [25]:
import numpy as np
from sklearn.decomposition import PCA

# 生成示例数据
np.random.seed(42)
data = np.random.rand(100, 10)

# 创建PCA对象，并拟合数据
pca = PCA()
pca.fit(data)

# 获取每个维度的解释度和累计方差解释率
explained_variance = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance)

print("每个维度的主成分(PC)值 : ")
for i in range(min(3,np.shape(data)[1])):
    print(f"维度 {i+1}: {pca.components_[i]}")

# 输出每个维度的解释度和累计方差解释率
print("\n每個维度的解釋度 : ")
for i, ev in enumerate(explained_variance):
    print(f"维度 {i+1}: {ev:.4f}")

print("\n累積方差解釋率 : ")
for i, cev in enumerate(cumulative_explained_variance):
    print(f"前 {i+1} 個维度累積方差解釋率：{cev:.4f}")

每个维度的主成分(PC)值 : 
维度 1: [-0.31644095  0.38832581  0.37263919 -0.08200572 -0.3611503   0.14104365
 -0.24575502  0.08832903  0.48643738  0.38515135]
维度 2: [-0.04233586  0.2798924   0.31062378 -0.14507819 -0.15805849 -0.49418697
 -0.18461535  0.45284893 -0.50331272 -0.20162139]
维度 3: [-0.02214896  0.37802299  0.11591659  0.36181336  0.59020145  0.2330573
 -0.49203875 -0.16012417 -0.20412184  0.014192  ]

每個维度的解釋度 : 
维度 1: 0.1611
维度 2: 0.1390
维度 3: 0.1180
维度 4: 0.1063
维度 5: 0.0968
维度 6: 0.0883
维度 7: 0.0859
维度 8: 0.0775
维度 9: 0.0679
维度 10: 0.0593

累積方差解釋率 : 
前 1 個维度累積方差解釋率：0.1611
前 2 個维度累積方差解釋率：0.3001
前 3 個维度累積方差解釋率：0.4181
前 4 個维度累積方差解釋率：0.5244
前 5 個维度累積方差解釋率：0.6212
前 6 個维度累積方差解釋率：0.7095
前 7 個维度累積方差解釋率：0.7953
前 8 個维度累積方差解釋率：0.8728
前 9 個维度累積方差解釋率：0.9407
前 10 個维度累積方差解釋率：1.0000


In [26]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# 生成模拟数据
X, y = make_regression(n_samples=100, n_features=10, noise=0.1, random_state=42)

# 使用PCA进行降维
pca = PCA()
X_pca = pca.fit_transform(X)

# 計算主成分的解釋度
explained_variance_ratio = pca.explained_variance_ratio_

# 計算累積解釋度
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# 找到累积解释方差大于等于0.8的主成分的数量
n_components = np.argmax(cumulative_explained_variance_ratio >= 0.8) + 1 #求累積解釋率 >= 0.8 的位置

# 取出累积解释方差大于等于0.8的主成分
X_pca_selected = X_pca[:, :n_components]

# 使用回归模型进行分析
regression_model = LinearRegression()
regression_model.fit(X_pca_selected, y)

# 输出模型的系数和截距
print("回歸模型系数：", regression_model.coef_)
print("回歸模型截距：", regression_model.intercept_)

回歸模型系数： [-80.06685648  84.98259857  45.76534356 -32.28268893  66.9524704
  68.45640835 -55.75366454]
回歸模型截距： 11.013216414049733


## AutoFeat : 升維

有些資料必須使用高維度去分析, 例如 y = X1^2 + X2^2

In [24]:
import numpy as np
import pandas as pd
from autofeat import AutoFeatRegressor

# 假設這是我們的資料
data = pd.DataFrame({
    'X1': [1, 2, 3, 4, 5],
    'X2': [2, 3, 4, 5, 6],
    'y': [5, 13, 25, 41, 61]
})

# 分割特徵和目標變數
X = data[['X1', 'X2']]
y = data['y']

# 建立 AutoFeatRegressor 物件
reg = AutoFeatRegressor()

# 使用 fit 函數進行特徵提升
X_transformed = reg.fit_transform(X, y)

# 取得新特徵名稱
new_feat_names = X_transformed.columns

print("原始資料形狀：", X.shape)
print("特徵提升後資料形狀：", X_transformed.shape)
print("原始特徵名稱：", X.columns)
print("新特徵名稱：", new_feat_names)


原始資料形狀： (5, 2)
特徵提升後資料形狀： (5, 3)
原始特徵名稱： Index(['X1', 'X2'], dtype='object')
新特徵名稱： Index(['X1', 'X2', 'exp(X2)'], dtype='object')


Recursive Feature Elimination (RFE)：RFE是一种递归特征消除方法，它通过反复训练模型并剔除最不重要的特征来选择特征。可以使用sklearn的RFE类来实现。

SelectKBest和SelectPercentile：这是一种基于统计检验的特征选择方法，它根据指定的统计指标（例如卡方检验、F检验、互信息等）选择排名前k个特征或排名前k%的特征。可以使用sklearn的SelectKBest和SelectPercentile类来实现。

SelectFromModel：这是一种基于模型的特征选择方法，它通过训练一个模型，并选择重要性高于指定阈值的特征。可以使用sklearn的SelectFromModel类来实现。

VarianceThreshold：这是一种简单的特征选择方法，它根据特征的方差来进行选择，只保留方差大于指定阈值的特征。可以使用sklearn的VarianceThreshold类来实现。

Boruta：Boruta是一种基于随机森林的特征选择方法，它通过比较原始特征和随机重排特征的重要性来选择特征。可以使用第三方库boruta来实现。